In [1]:
import sentier_data_tools as sdt
import sentier_peakachu as sp
from datetime import datetime

In [2]:
sp.create_local_electricity_datastorage()

/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/utils_location.py:40: UserWarning: ISO code ABW not found
  warnings.warn(f"ISO code {iso_code} not found")
/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/data.py:129: UserWarning: Location not found for ABW, skipping Dataset creation
  warnings.warn(
/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/data.py:122: UserWarning: Source type gas, other_fossil not found, skipping Dataset creation
  warnings.warn(
/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/data.py:122: UserWarning: Source type gas, oil not found, skipping Dataset creation
  warnings.warn(
/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/data.py:122: UserWarning: Source type gas, oil, other_fossil not found, skipping Dataset creation
  warnings.warn(
/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/data.py:122: UserWarning: Source type gas, other_

## ElectricitySourceModel

In [3]:
el_from_gas = sdt.ProductIRI("https://vocab.sentier.dev/products/electricity/fossil-gas")
location = sdt.GeonamesIRI(sp.get_geonames_iri_from_iso_code("DE"))
demand = sdt.Demand(
    product_iri=el_from_gas,
    unit_iri=sdt.UnitIRI("https://vocab.sentier.dev/units/unit/KilowattHour"),
    amount=10,
    spatial_context=location,
    begin_date=datetime(2022, 10, 8, 6, 49, 0),
    end_date=datetime(2022, 10, 9, 6, 59, 0),
)
model = sp.ElectricitySourceModel(demand=demand, run_config=sdt.RunConfig())
plant_emissions = model.get_model_data()
model.calculate_average_source_emission_factor(plant_emissions)

0.47760833563345295

## ElectricityMixModel

In [4]:
generic_electricity = sdt.ProductIRI("http://openenergy-platform.org/ontology/oeo/OEO_00000139") # electrical energy
location = sdt.GeonamesIRI(sp.get_geonames_iri_from_iso_code("DE"))
demand = sdt.Demand(
    product_iri=generic_electricity,
    unit_iri=sdt.UnitIRI("https://vocab.sentier.dev/units/unit/KilowattHour"),
    amount=10,
    spatial_context=location,
    begin_date=datetime(2022, 10, 8, 3, 46, 0),
    end_date=datetime(2022, 10, 8, 3, 49, 0),
)
model = sp.ElectricityMixModel(demand=demand, run_config=sdt.RunConfig())
df = model.get_model_data()

In [5]:
mix = model.calculate_market_mix(df)

In [6]:
model.calculate_impact(mix)

2024-10-11 14:02:09.129 | DEBUG    | sentier_peakachu.model:get_bonsai_emission_factor:175 - http://openenergyplatform.org/ontology/oeo/OEO_00010417/: 0    0.003956
Name: https://example.com/indirect_CO2_emissions, dtype: float64 kgCO2eq/kWh
/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/model.py:176: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(total_emissions)
2024-10-11 14:02:09.132 | DEBUG    | sentier_peakachu.model:get_bonsai_emission_factor:175 - http://openenergyplatform.org/ontology/oeo/OEO_00010419/: 0    0.005862
Name: https://example.com/indirect_CO2_emissions, dtype: float64 kgCO2eq/kWh
/Users/timodiepers/Documents/Coding/sentier_peakachu/sentier_peakachu/model.py:176: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(total_emissions

6.648689682699809